<a href="https://colab.research.google.com/github/AdamJelley/AdventOfCode2021/blob/main/Day22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Day 22

In [138]:
import requests
import numpy as np
from tqdm import tqdm

In [139]:
#Variables to set
day = 22
cookie = ''

In [140]:
input_link = f'https://adventofcode.com/2021/day/{day}/input'
user_cookie = {'session':cookie} # Retrieve session cookie corresponding to user login (by inspecting cookies on data page)
raw_input = requests.get(input_link, cookies=user_cookie).text[:-1]

In [141]:
processed_input = raw_input.split('\n')

In [147]:
switch = [int(instruction.split(' ')[0].replace('on','1').replace('off','0')) for instruction in processed_input]
cuboids = np.array([[list(map(int, coordinate.replace('x=','').replace('y=','').replace('z=','').split('..'))) for coordinate in instruction.split(' ')[1].split(',')] for instruction in processed_input])

## Part 1

In [7]:
initialisation_reactor = np.zeros((101, 101, 101), dtype=int)

for i, coordinates in enumerate(cuboids):
  if np.all(coordinates<50) and np.all(-50<coordinates):
    [[x1, x2], [y1,y2], [z1, z2]] = coordinates+50
    initialisation_reactor[x1:x2+1, y1:y2+1, z1:z2+1] = switch[i]

In [8]:
print(f'Number of lit cubes = {initialisation_reactor.sum()}')

Number of lit cubes = 577205


## Part 2

In [53]:
cuboids[:,:,1]+=1

In [ ]:
volume=np.uint(0)
x_sorted = np.sort(cuboids[:,0,:].flatten())
y_sorted = np.sort(cuboids[:,1,:].flatten())
z_sorted = np.sort(cuboids[:,2,:].flatten())
cuboids_reversed = cuboids[:][::-1]
switch_reversed = switch[:][::-1]
reversed_cuboids_switches = list(zip(cuboids_reversed, switch_reversed))

for (x, x_next, y, y_next, z, z_next) in tqdm(((x, x_next, y, y_next, z, z_next) for x, x_next in zip(x_sorted[:-1], x_sorted[1:]) for y, y_next in zip(y_sorted[:-1], y_sorted[1:]) for z, z_next in zip(z_sorted[:-1], z_sorted[1:]))):
  for cuboid, state in reversed_cuboids_switches:
    if cuboid[0][0] <= x < cuboid[0][1] and cuboid[1][0] <= y < cuboid[1][1] and cuboid[2][0] <= z < cuboid[2][1]:
      if state==1:
        volume+=(x_next-x)*(y_next-y)*(z_next-z)
      break
          
print('\n')
print(f'Total number of lit cubes: {int(volume)}')

Total number of lit cubes: 1197308251666843

## Part 2 Original Attempt

In [10]:
assert np.max(cuboids)<100000
assert np.min(cuboids)>-100000

In [35]:
def calculate_volume(cubes):
  total_volume = 0
  for cube in cubes:
    [[x1, x2], [y1, y2], [z1, z2]] = cube
    volume = (x2+1-x1)*(y2+1-y1)*(z2+1-z1)
    total_volume+=volume
  return total_volume

In [78]:
def calculate_overlap(cube1, cube2):

  [[x1, x2], [y1, y2], [z1, z2]] = cube1
  [[fut_x1, fut_x2], [fut_y1, fut_y2], [fut_z1, fut_z2]] = cube2

  if (fut_x1 <= x1 <= fut_x2 or fut_x1 <= x2 <= fut_x2) \
  and (fut_y1 <= y1 <= fut_y2 or fut_y1 <= y2 <= fut_y2) \
  and (fut_z1 <= z1 <= fut_z2 or fut_z1 <= z2 <= fut_z2):
    overlap = (min(x2+1, fut_x2+1) - max(x1, fut_x1))*(min(y2+1, fut_y2+1) - max(y1, fut_y1))*(min(z2+1, fut_z2+1) - max(z1, fut_z1))
    overlap_cube = [[max(x1, fut_x1), min(x2, fut_x2)],[max(y1, fut_y1), min(y2, fut_y2)],[max(z1, fut_z1), min(z2, fut_z2)]]
    new_cubes = [[[x1+1, x2-1], [y1+1, y2-1], [z1, overlap_cube[2][0]-1]], 
                 [[x1+1, x2-1], [y1+1, y2-1], [overlap_cube[2][1]+1, z2]], 
                 [[overlap_cube[0][0]+1, x2-1], [y1, overlap_cube[1][0]-1], [overlap_cube[2][0]+1, overlap_cube[2][1]-1]], 
                 [[overlap_cube[0][1]+1, x2-1], [overlap_cube[1][0]+1, y2-1], [overlap_cube[2][0]+1, overlap_cube[2][1]-1]], 
                 [[x1, overlap_cube[0][1]-1], [overlap_cube[1][1]+1, y2-1], [overlap_cube[2][0]+1, overlap_cube[2][1]-1]], 
                 [[x1, overlap_cube[0][0]-1], [y1, overlap_cube[1][1]-1], [overlap_cube[2][0]+1, overlap_cube[2][1]-1]]]
    return overlap, overlap_cube, new_cubes
  else:
    overlap = 0
    return overlap, None, None                                                                                 

In [79]:
overlap, overlap_cube, new_cubes = calculate_overlap([[1,2],[1,2],[1,2]], [[2,3],[2,3],[2,3]])
new_cubes

[[[2, 1], [2, 1], [1, 1]],
 [[2, 1], [2, 1], [3, 2]],
 [[3, 1], [1, 1], [3, 1]],
 [[3, 1], [3, 1], [3, 1]],
 [[1, 1], [3, 1], [3, 1]],
 [[1, 1], [1, 1], [3, 1]]]

Allowing for overlap of multiple cubes is getting complex. Approach would be to split all cubes into non-overlapping cubes and sum volumes, but would take time to implement. Instead enjoy break and leave more brute force coordinate compression approach running!